In [ ]:
import pandas as pd
import numpy as np
import os
import re
import time
import datetime
import gc
import string
import re
import seaborn as sns 
import matplotlib.pyplot as plt 
import math

from decimal import Decimal, getcontext

# getcontext().prec = 9
# pd.set_option("display.precision", 9)

m = 1
N = 2*50
kB = 1
Temp_Target = 1

Partic_DF=pd.read_csv(r"E:\Cristian\1. Molecular Dinamics\2. Python Projects\3. Jupiter Code\3. Test\Analitica Nosee Hoover IG\Particle_Data.csv")

System_DF=pd.read_csv(r"E:\Cristian\1. Molecular Dinamics\2. Python Projects\3. Jupiter Code\3. Test\Analitica Nosee Hoover IG\System_Data.csv")


In [ ]:
def obtener_columnas_object(dataframe):
    columnas_object = []
    tipos_datos = dataframe.dtypes
    for columna, tipo in tipos_datos.items():
        if tipo == 'object':
            columnas_object.append(columna)
    return columnas_object

# -------- Conversión de data types y alistamiento de dataframes originales --------

columns_to_convert = obtener_columnas_object(System_DF)
# print(columns_to_convert)
System_DF[columns_to_convert] = System_DF[columns_to_convert].replace(',', '.', regex=True)
System_DF[columns_to_convert] = System_DF[columns_to_convert].astype(float)
# print(System_DF.dtypes)

columns_to_convert = obtener_columnas_object(Partic_DF)
# print(columns_to_convert)
Partic_DF[columns_to_convert] = Partic_DF[columns_to_convert].replace(',', '.', regex=True)
Partic_DF[columns_to_convert] = Partic_DF[columns_to_convert].astype(float)
# print(Partic_DF.dtypes)


In [ ]:
# Calcula la suma de 'Ke_Energy' para cada valor único en 'Step'
grouped_sum = Partic_DF.groupby('Step')['Ke_Energy'].sum()

# Asigna la suma correspondiente a cada fila en 'Calo_Cap'
Partic_DF['Ke_Total'] = Partic_DF['Step'].map(grouped_sum)

Partic_DF['Ke_Mean'] = Partic_DF['Vel_M']**2
Partic_DF['Ke_Mean'] = Partic_DF.groupby('Step')['Ke_Mean'].transform('mean')
Partic_DF['Ke_Mean'] = Partic_DF['Ke_Mean']*m*0.5


Const = 1/(N*kB*(Temp_Target**2))
# Partic_DF['Cap_Cal'] = Const * () Partic_DF['Ke_Mean']*m*0.5

Partic_DF['Cap_Cal'] = Partic_DF['Ke_Total']**2
Partic_DF['Cap_Cal'] = Partic_DF.groupby('Step')['Cap_Cal'].transform('mean')

Partic_DF['Cap_Cal'] = Const * ((Partic_DF['Cap_Cal'])-((System_DF['Ke_T'].mean()**2)))

# # Agrupa por 'Step' y asigna el cálculo correspondiente a cada fila en la columna 'Columna_Nueva'
# Partic_DF['Ke_Mean'] = Partic_DF.groupby('Step')['Vel_M'**2].transform('mean')

# # Partic_DF['Ke_Mean'] = 0.5*m*Partic_DF['Ke_Mean']